# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [1]:
# try this project here!

In [1]:
import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

W0709 23:22:58.487702 4467238336 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/anaconda3/envs/pysyft/lib/python3.6/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0709 23:22:58.518723 4467238336 deprecation_wrapper.py:119] From /anaconda3/envs/pysyft/lib/python3.6/site-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')
secure_worker = sy.VirtualWorker(hook, id='secure_worker')

In [3]:
bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

W0709 23:22:59.483520 4467238336 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0709 23:22:59.487669 4467238336 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0709 23:22:59.491333 4467238336 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0709 23:22:59.494824 4467238336 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0709 23:22:59.497965 4467238336 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0709 23:22:59.510921 4467238336 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [4]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [5]:
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

In [6]:
alices_data = data[2:4].send(alice)
alices_target = target[2:4].send(alice)

In [7]:
# Initialize A toy model
model = nn.Linear(2,1)

In [8]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

In [9]:
bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

In [10]:
for i in range(10):
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_target - bobs_pred)**2).sum()
    bobs_loss.backward()

    # we update the gradients
    bobs_opt.step()
    bobs_loss = bobs_loss.get().data
    print(bobs_loss)

    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_target - alices_pred)**2).sum()
    alices_loss.backward()

    # we update the gradients
    alices_opt.step()
    alices_loss = alices_loss.get().data
    print(alices_loss)

tensor(1.8248)
tensor(0.0761)
tensor(0.1489)
tensor(0.0619)
tensor(0.1134)
tensor(0.0521)
tensor(0.0943)
tensor(0.0443)
tensor(0.0785)
tensor(0.0378)
tensor(0.0653)
tensor(0.0322)
tensor(0.0544)
tensor(0.0275)
tensor(0.0452)
tensor(0.0234)
tensor(0.0377)
tensor(0.0200)
tensor(0.0313)
tensor(0.0170)


## Averaging Bob and Alice together

In [11]:
alices_model.move(secure_worker)
bobs_model.move(secure_worker)

In [12]:
secure_worker._objects

{98037206720: Parameter containing:
 tensor([[0.1660, 0.1518]], requires_grad=True),
 47580313037: Parameter containing:
 tensor([-0.1026], requires_grad=True),
 62265510766: Parameter containing:
 tensor([[0.2267, 0.6441]], requires_grad=True),
 42167916311: Parameter containing:
 tensor([0.2286], requires_grad=True)}

In [15]:
# Now we average them together
model.weight.data.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())

RuntimeError: a leaf Variable that requires grad has been used in an in-place operation.

In [16]:
model.bias.data.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

RuntimeError: set_storage is not allowed on Tensor created from .data or .detach()

In [17]:
with th.no_grad():

    model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

## Rinse and repeat

In [18]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):

    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

    for wi in range(worker_iters):
        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target) ** 2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data

    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)

    with th.no_grad():

        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0220) Alice:tensor(0.0038)
Bob:tensor(0.0105) Alice:tensor(0.0008)
Bob:tensor(0.0048) Alice:tensor(0.0002)
Bob:tensor(0.0023) Alice:tensor(0.0003)
Bob:tensor(0.0011) Alice:tensor(0.0004)
Bob:tensor(0.0005) Alice:tensor(0.0006)
Bob:tensor(0.0003) Alice:tensor(0.0006)
Bob:tensor(0.0001) Alice:tensor(0.0005)
Bob:tensor(7.6028e-05) Alice:tensor(0.0005)
Bob:tensor(4.3006e-05) Alice:tensor(0.0004)


In [19]:
preds = model(data)
loss = ((preds - target)**2).sum()

In [20]:
preds

tensor([[0.9213],
        [0.9293],
        [0.0554],
        [0.0634]], grad_fn=<AddmmBackward>)

In [21]:
target

tensor([[1.],
        [1.],
        [0.],
        [0.]], requires_grad=True)

In [23]:
loss.data

tensor(0.0183)

# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [13]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [14]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [15]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [16]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [17]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [18]:
(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [19]:
x_share = (2,5,7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [ ]:
# try this project here!

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [25]:
BASE=10
PRECISION=4

In [26]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [27]:
encode(3.5)

35000

In [28]:
decode(35000)

3.5

In [29]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

7.8

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [30]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [31]:
x = th.tensor([1,2,3,4,5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [32]:
x = x.share(bob, alice, secure_worker)

In [33]:
bob._objects

{35498656553: tensor([  10235770278698899, 1401398179551373756, 2277280072169145491,
          636965538565031298,  913795591610271305])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [34]:
y = x + x

In [35]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:23637986557 -> bob:30254176063]
	-> (Wrapper)>[PointerTensor | me:18229131498 -> alice:75856222543]
	-> (Wrapper)>[PointerTensor | me:34301722959 -> secure_worker:75419815101]
	*crypto provider: me*

In [36]:
y.get()

tensor([ 2,  4,  6,  8, 10])

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [37]:
x = th.tensor([0.1,0.2,0.3])

In [38]:
x

tensor([0.1000, 0.2000, 0.3000])

In [39]:
x = x.fix_prec()

In [40]:
x.child.child

tensor([100, 200, 300])

In [41]:
y = x + x

In [42]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

### Shared Fixed Precision

And of course, we can combine the two!

In [43]:
x = th.tensor([0.1, 0.2, 0.3])

In [44]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [45]:
y = x + x

In [46]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation